In [1]:
import random
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.preprocessing import MinMaxScaler
from sklearn.svm import SVR
from sklearn.metrics import mean_absolute_percentage_error
from deap import base, creator, tools, algorithms

In [2]:
data = pd.read_csv('DATA_DESA.csv')

df = pd.DataFrame(data)

# Normalisasi MinMax
scaler = MinMaxScaler()
df['jumlah_kecelakaan'] = scaler.fit_transform(df[['jumlah_kecelakaan']])

# Menambahkan kolom lag (pergeseran)
df['jumlah_kecelakaan_lag'] = df['jumlah_kecelakaan'].shift(1)
df = df.dropna() # Menghapus baris dengan nilai NaN

In [3]:
# Memisahkan data latih dan data uji sesuai rasio (0.8 untuk 80:20)
train_size = int(0.7 * len(df))
train_data = df.iloc[:train_size]
test_data = df.iloc[train_size:]

# Memisahkan fitur dan target untuk data latih
X_train = train_data['jumlah_kecelakaan_lag'].values.reshape(-1, 1)
y_train = train_data['jumlah_kecelakaan'].values

# Memisahkan fitur dan target untuk data uji
X_test = test_data['jumlah_kecelakaan_lag'].values.reshape(-1, 1)
y_test = test_data['jumlah_kecelakaan'].values

In [4]:
# Define evaluation function
def evaluate(individual):
    C, epsilon, gamma = individual
    if C <= 0 or epsilon <= 0 or gamma <= 0:
        return float('inf'),
    svr = SVR(C=C, epsilon=epsilon, gamma=gamma)
    svr.fit(X_train, y_train)
    y_pred = svr.predict(X_test)
    mape = mean_absolute_percentage_error(y_test, y_pred) * 100
    return mape,
    
# Setup DEAP framework
creator.create("FitnessMin", base.Fitness, weights=(-1.0,))
creator.create("Individual", list, fitness=creator.FitnessMin)

toolbox = base.Toolbox()

# Parameter ranges
C_range = (10, 50)
epsilon_range = (0.001, 5)
gamma_range = (0.001, 5)

def create_individual():
    C = random.uniform(*C_range)
    epsilon = random.uniform(*epsilon_range)
    gamma = random.uniform(*gamma_range)
    return creator.Individual([C, epsilon, gamma])
    
toolbox.register("individual", create_individual)
toolbox.register("population", tools.initRepeat, list, toolbox.individual)

toolbox.register("mate", tools.cxUniform, indpb=0.5)  # Menggunakan crossover uniform 
toolbox.register("mutate", tools.mutGaussian, mu=0, sigma=1, indpb=0.2) # Menggunakan mutasi gaussian
toolbox.register("select", tools.selTournament, tournsize=3) # Menggunakan seleksi turnamen
toolbox.register("evaluate", evaluate)

In [5]:
# Initialize population
population_size = 300
population = toolbox.population(n=population_size)

# Genetic algorithm parameters    
num_generations = 30
crossover_rate = 0
mutation_rate = 1

# Run genetic algorithm
for generation in range(num_generations):
    print(f"Generasi {generation + 1}:")
    offspring = algorithms.varAnd(population, toolbox, cxpb=crossover_rate, mutpb=mutation_rate)
    for ind in offspring:
        ind[0] = max(min(ind[0], C_range[1]), C_range[0])
        ind[1] = max(min(ind[1], epsilon_range[1]), epsilon_range[0])
        ind[2] = max(min(ind[2], gamma_range[1]), gamma_range[0])
        
    fits = toolbox.map(toolbox.evaluate, offspring)
    for fit, ind in zip(fits, offspring):
        ind.fitness.values = fit
        print(f"Individu: {ind}, MAPE: {fit[0]}")
    
    population = toolbox.select(offspring, k=len(population))

# Get the best individual
best_individual = tools.selBest(population, k=1)[0]
best_mape = best_individual.fitness.values[0]
C, epsilon, gamma = best_individual

# Test the best SVR model
best_svr = SVR(kernel='rbf', C=C, epsilon=epsilon, gamma=gamma)
best_svr.fit(X_train, y_train)
y_pred = best_svr.predict(X_test)
mape = mean_absolute_percentage_error(y_test, y_pred) * 100

print("\nParameter terbaik:")
print(f"C: {C}")
print(f"Epsilon: {epsilon}")
print(f"Gamma: {gamma}")
print(f"MAPE: {mape:.4f}")

Generasi 1:
Individu: [30.63980450228726, 4.713512801534581, 4.856525641543247], MAPE: 49.03560405882388
Individu: [45.17427773833361, 3.8887318901176515, 3.5064724867736072], MAPE: 49.03560405882388
Individu: [12.779464517118697, 3.289446755442559, 1.5137974239799807], MAPE: 49.03560405882388
Individu: [41.378869433197984, 4.245556714631445, 1.0742658715763074], MAPE: 49.03560405882388
Individu: [17.049320933332126, 1.3978354106537383, 4.312096307032362], MAPE: 49.03560405882388
Individu: [16.724593263098516, 1.5559463612429345, 0.535966962306194], MAPE: 49.03560405882388
Individu: [47.38272921545207, 0.919244161322102, 1.8268895952719342], MAPE: 49.03560405882388
Individu: [28.14817579196572, 1.628263101202363, 2.010955987759008], MAPE: 49.03560405882388
Individu: [16.996246835891768, 0.769290101858099, 1.4492059481342916], MAPE: 49.03560405882388
Individu: [35.50656275479963, 4.941050928200374, 3.262546477854325], MAPE: 49.03560405882388
Individu: [41.47361977885168, 2.4571825558797

In [6]:
# Prediksi 6 bulan ke depan
last_value = df['jumlah_kecelakaan'].values[-1] # Nilai terakhir dari data sebenarnya
predicts = []

for i in range(6):
    next_value = best_svr.predict(np.array([[last_value]]))[0]
    predicts.append(next_value)
    last_value = next_value

# Denormalisasi prediksi
predicts = scaler.inverse_transform(np.array(predicts).reshape(-1, 1))

print("Prediksi untuk 6 bulan ke depan:")
print(predicts)

Prediksi untuk 6 bulan ke depan:
[[11.07729839]
 [10.33635057]
 [10.04006654]
 [ 9.93925197]
 [ 9.90682486]
 [ 9.896575  ]]
